In [1]:
#importing pandas library
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CONFIRMED,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [3]:
# Remove Space for `FALSE POSITIVE` category
mask = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[mask, "koi_disposition"] = "False_Positive"
df["koi_disposition"].head()

0         CONFIRMED
1         CONFIRMED
2    False_Positive
3    False_Positive
4         CONFIRMED
Name: koi_disposition, dtype: object

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
from sklearn.model_selection import train_test_split
y = df["koi_disposition"]
X = df.drop(columns=["koi_disposition"])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Pre-processing

Scale the data using LabelEncoder and MinMaxScaler

In [5]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

In [7]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


# Create and train the Deep Learning Model using Keras

In [8]:
# An empty model
model = Sequential()

# adding input layers
model.add(Dense(units=50, activation='relu', input_dim=40))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=50, activation='relu'))


# Add outpul layer
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                2050      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 153       
Total params: 7,303
Trainable params: 7,303
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X_train_scaled, y_train_categorical, epochs=50, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 1s - loss: 0.5749 - acc: 0.7030
Epoch 2/50
 - 0s - loss: 0.3794 - acc: 0.8024
Epoch 3/50
 - 0s - loss: 0.3620 - acc: 0.8164
Epoch 4/50
 - 0s - loss: 0.3532 - acc: 0.8181
Epoch 5/50
 - 0s - loss: 0.3477 - acc: 0.8278
Epoch 6/50
 - 0s - loss: 0.3407 - acc: 0.8310
Epoch 7/50
 - 0s - loss: 0.3308 - acc: 0.8397
Epoch 8/50
 - 0s - loss: 0.3287 - acc: 0.8452
Epoch 9/50
 - 0s - loss: 0.3206 - acc: 0.8498
Epoch 10/50
 - 0s - loss: 0.3150 - acc: 0.8535
Epoch 11/50
 - 0s - loss: 0.3090 - acc: 0.8600
Epoch 12/50
 - 0s - loss: 0.3065 - acc: 0.8618
Epoch 13/50
 - 0s - loss: 0.3046 - acc: 0.8643
Epoch 14/50
 - 0s - loss: 0.3091 - acc: 0.8588
Epoch 15/50
 - 0s - loss: 0.2958 - acc: 0.8669
Epoch 16/50
 - 0s - loss: 0.2938 - acc: 0.8713
Epoch 17/50
 - 0s - loss: 0.2900 - acc: 0.8713
Epoch 18/50
 - 0s - loss: 0.2911 - acc: 0.8719
Epoch 19/50
 - 0s - loss: 0.2911 - acc: 0.8679
Epoch 20/50
 - 0s - loss: 0.2823 - acc: 0.8774
Epoch 21/50
 - 0s - l

In [12]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 0s - loss: 0.2815 - acc: 0.8838
Normal Neural Network - Loss: 0.2815291029209406, Accuracy: 0.883806049823761


In [13]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [14]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['False_Positive' 'CONFIRMED' 'CONFIRMED' 'CANDIDATE' 'CONFIRMED']
Actual Labels: ['False_Positive', 'CANDIDATE', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE']
